## **Supply Usage Analysis**

### **Project Overview**
This project analyzes supply usage patterns across retail and warehouse channels using the Warehouse and Retail Sales dataset from [Data.gov](https://data.gov).  
The goal is to uncover insights about product movement, supplier performance, and seasonal trends thereby helping inform smarter inventory management and demand planning decisions. The framework presented herein can be easily adapted to other inventory datasets.

---

### **Objectives**
- Combine retail and warehouse data to create a unified supply usage metric  
- Explore trends over time by supplier, item type, and sales channel  
- Identify top-moving products and periods of unusually high or low demand  
- Visualize patterns and provide data-driven recommendations for inventory optimization  

---

### **Data**
This dataset includes monthly records of warehouse and retail activity with the following key fields:
- `year`, `month` - reporting period  
- `supplier`, `item_code`, `item_description`, `item_type` - product identifiers  
- `retail_sales`, `retail_transfers`, `warehouse_sales` - quantities across distribution channels  
- `date` - constructed timestamp for time-series analysis  

Data Source: [Data.gov - Warehouse and Retail Sales Dataset](https://data.gov)

---

### **Workflow**
1. **Data Cleaning**:  handle missing values, create a monthly `date` column  
2. **Feature Engineering**:  calculate a total `usage` metric combining all outflow channels  
3. **Exploratory Analysis**:  aggregate and visualize usage trends  
4. **Insights**: summarize supplier performance, seasonal effects, and usage variability  
  
---

### **Skills Demonstrated**

- Data wrangling & cleaning (pandas)
- Feature engineering
- Time Series Analysis
- Exploratory Data Analysis (EDA)
- Data Visualization
- Business-Oriented Analytics
- Reproducible Analysis

---

*Created by [Graham Kesler O'Connor](https://github.com/grahamkesler)*  
*Project Type: Exploratory Data Analysis (EDA) | Language: Python | Dataset: Public (Data.gov)*  


In [ ]:
# Load necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Loading the Warehouse & Retail Sales Dataset

This project begins by loading the Warehouse and Retail Sales dataset directly from a public API endpoint. Instead of downloading a CSV manually, the notebook retrieves the data programmatically from Montgomery County's open data portal, ensuring the workflow remains fully reproducible and always reflects the most current version of the dataset.

At a high level, the code below:
1. Specifies the API endpoint that returns the dataset in CSV format  
2. Uses `pandas.read_csv()` to fetch and parse the data directly into a DataFrame  
3. Displays an initial preview to confirm the structure and validate that the dataset loaded correctly  

Starting with an automated data-ingestion step mirrors modern data science workflows and sets the foundation for the cleaning, feature engineering, and analysis that follow.



In [ ]:
# URL of downloadable CSV file for the Warehouse and Retail Sales dataset from Data.gov
data_url = "https://data.montgomerycountymd.gov/api/views/v76h-r7br/rows.csv?accessType=DOWNLOAD"

# Read the dataset into a pandas DataFrame
df = pd.read_csv(data_url)

# Overview of column types and non-null counts
df.info()

# Display the first few rows
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              307645 non-null  int64  
 1   MONTH             307645 non-null  int64  
 2   SUPPLIER          307478 non-null  object 
 3   ITEM CODE         307645 non-null  object 
 4   ITEM DESCRIPTION  307645 non-null  object 
 5   ITEM TYPE         307644 non-null  object 
 6   RETAIL SALES      307642 non-null  float64
 7   RETAIL TRANSFERS  307645 non-null  float64
 8   WAREHOUSE SALES   307645 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 21.1+ MB


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0


### Data Cleaning and Quality Audit

Before diving into analysis, we first establish a clean and reliable dataset. The code below constructs a proper date field, standardizes column names, and performs basic checks for duplicates and missing values. These are foundational steps in any analytical workflow and ensure that subsequent results are trustworthy and reproducible.

At a high level, this section:
- Creates a unified monthly `date` column to support time series analysis  
- Normalizes column names for consistency and easier downstream referencing  
- Identifies duplicate records and quantifies missingness across fields  
- Produces a compact audit table summarizing where data gaps exist   

Together, these operations provide a clear understanding of the dataset's quality and prepare it for feature engineering, aggregation, and modeling in later steps.


In [ ]:
# create date column
df['date'] = pd.to_datetime(dict(year=df['YEAR'],month=df['MONTH'], day = 1))

# reformat column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

# number of unique values in each column
unique_counts = df.nunique()

# number of duplicate rows
dups = df.duplicated().astype(int).sum()
print("Number of Duplicated Rows:",dups)

# find number and percent of missing entries for each column
missing_cnt = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean()*100).round(2).sort_values(ascending=False)

audit = pd.DataFrame({"unique_n":unique_counts,"missing_n": missing_cnt, "missing_%": missing_pct})
print(audit)

Number of Duplicated Rows: 0
                  unique_n  missing_n  missing_%
date                    24          0       0.00
item_code            34056          0       0.00
item_description     34822          0       0.00
item_type                8          1       0.00
month                   12          0       0.00
retail_sales         10674          3       0.00
retail_transfers      2504          0       0.00
supplier               396        167       0.05
warehouse_sales       4895          0       0.00
year                     4          0       0.00


### Calculating Total Usage and Reshaping Flow Data

Next we create a consolidated measure of product movement and restructure the dataset so usage can be analyzed from multiple angles. By combining flow channels and organizing the data in both long and wide formats, we build a foundation that supports flexible trend analysis, item-level comparisons, and deeper diagnostic checks.

---

##### **1. Create a total usage metric**

Outflow measures such as retail sales, transfers, and warehouse activity are combined into a single `usage` variable. This unified metric provides a clear view of overall demand and allows for meaningful comparisons across suppliers, items, and time periods.


##### **2. Restructure the data into a long format**

The dataset is reshaped from wide to long format so each flow type becomes its own row. This tidy structure makes it easier to analyze individual channels, create grouped summaries, and build clearer visualizations that highlight how usage is distributed across flow types.


##### **3. Summarize flow-type activity across key dimensions**

Using the long format, usage is aggregated by date, supplier, item, and flow category to produce a clean breakdown of how much movement occurred in each channel. Pivoting these summaries back to a wide table provides a straightforward view of the contribution of each flow type.


##### **4. Aggregate monthly usage**

Finally, total usage is rolled up to a monthly level. Monthly aggregation smooths day-to-day noise and highlights broader patterns useful for identifying seasonality, monitoring trends, and preparing data for forecasting or planning analyses.

---

Together, these transformations create an analysis ready dataset that supports both high level demand insights and detailed exploration of product movement across channels.


In [ ]:
# Define total usage as the sum of all outflow channels
FLOW_COLS = ['retail_sales', 'retail_transfers', 'warehouse_sales']
# if columns aren't already numeric
#df[FLOW_COLS] = df[FLOW_COLS].apply(pd.to_numeric, errors='coerce')
df['usage'] = df[FLOW_COLS].sum(axis=1).fillna(0)

# Keep a tidy view of each flow for breakdowns
flows_long = df.melt(
    id_vars=['date', 'supplier','item_code','item_description','item_type'],
    value_vars=FLOW_COLS,
    var_name='flow_type',
    value_name='qty'
)

# flow breakdown
flow_breakdown = (
    flows_long
      .groupby(['date','supplier','item_code','flow_type'], as_index=False)['qty']
      .sum()
      .pivot_table(index=['date','supplier','item_code'], columns='flow_type', values='qty', fill_value=0)
      .reset_index()
)

# Aggregate monthly usage acrocss item type
monthly = (
    df.groupby(['item_type',
                pd.Grouper(key='date',freq='MS')],as_index=False)
    .agg(usage=('usage','sum'))
    .sort_values(['item_type','date'])
)


### Adding Trend and Time-Based Features

To understand how usage evolves over time, this section creates several trend-focused features commonly used in time-series and supply analytics. These engineered metrics make it easier to spot meaningful shifts in demand, smooth variability, and compare current behavior to historical patterns.

---

##### **1. Compute month-over-month (MoM) change**

Usage values are compared to the previous month for each supplier and item combination.  
This highlights short term demand changes and helps surface sharp increases, declines, or volatility that may warrant deeper investigation.


##### **2. Compute a 3-month rolling average**

A 3-month moving average is calculated to smooth out short term fluctuations and reveal underlying demand patterns.  
This is useful for stabilizing noisy usage data and preparing features for downstream forecasting models.


##### **3. Compute year-to-date (YTD) usage**

Cumulative usage is calculated within each calendar year for every supplier and item pair.  
This metric supports operational reporting by showing how annual consumption is tracking relative to expectations or historical norms.

---

Together, these trend features create a richer and more interpretable dataset that supports clearer pattern recognition, more robust visualizations, and stronger modeling or anomaly detection capabilities.


In [ ]:
# adding trend features

# add missing months
full_dates = pd.date_range(
    start=monthly['date'].min(),
    end=monthly['date'].max(),
    freq='MS'
)

item_types = monthly['item_type'].unique()

full_grid = (
    pd.MultiIndex.from_product(
        [item_types, full_dates],
        names=['item_type', 'date']
    )
    .to_frame(index=False)
)

monthly_filled = (
    full_grid
    .merge(monthly, on=['item_type','date'], how='left')
    .sort_values(['item_type','date'])
)

# Per-item MoM change

monthly_filled['usage_mom'] = (
    monthly_filled
      .groupby(['item_type'])['usage']
      .pct_change(fill_method=None)
)

# Per-item rolling 3-month average
monthly_filled['usage_ma3'] = (
    monthly_filled
      .groupby(['item_type'])['usage']
      .rolling(3, min_periods=1).mean()
      .reset_index(level=[0,1], drop=True)
)

# Per-item rolling YTD totals
monthly['year'] = monthly['date'].dt.year
monthly['ytd_usage'] = (
    monthly
      .groupby(['item_type','year'])['usage']
      .cumsum()
)

In [ ]:
fig = px.line(
    monthly,
    x='date',
    y='usage',
    color='item_type',
    markers=True,
    title='Monthly Usage by Item Type'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Usage',
    legend_title='Item Type'
)

fig.show()

fig = px.line(
    monthly_filled,
    x='date',
    y='usage_mom',
    color='item_type',
    markers=True,
    title='Month-over-Month (MoM) Usage Change by Item Type'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='MoM % Change',
    legend_title='Item Type'
)

fig.show()

fig = px.line(
    monthly_filled,
    x='date',
    y='usage_ma3',
    color='item_type',
    markers=True,
    title='Rolling 3-month average Usage by Item Type'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='3-month rolling average Usage',
    legend_title='Item Type'
)

fig.show()

fig = px.line(
    monthly,
    x='date',
    y='ytd_usage',
    color='item_type',
    markers=True,
    title='Rolling YTD Usage by Item Type'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='YTD Usage',
    legend_title='Item Type'
)

fig.show()

### Identifying the Top Moving Items in the Latest Month

Next we identify which items have the highest usage in the most recent month of available data.

At a high level, the code:
1. Determines the latest month present in the dataset  
2. Filters the data to that month to create a current snapshot  
3. Selects the top 10 items based on total usage  

The output provides a focused view of high velocity items, supporting quick assessments of demand hotspots and informing short-term stocking or procurement decisions.


In [ ]:
# top moving items from previous month
last_month = monthly['date'].max()
df_last_month = df[df['date']==last_month]
top_items = df.groupby(['item_code','item_type','item_description','supplier'],as_index=False)['usage'].sum().nlargest(10,'usage')
top_items

,item_code,item_type,item_description,supplier,usage
4948,23445,BEER,CORONA EXTRA LOOSE NR - 12OZ,CROWN IMPORTS,352574.83
37998,96750,BEER,CORONA EXTRA 2/12 NR - 12OZ,CROWN IMPORTS,266992.08
6885,23886,BEER,HEINEKEN LOOSE NR - 12OZ,HEINEKEN USA,206675.17
38015,96970,BEER,HEINEKEN 2/12 NR - 12OZ,HEINEKEN USA,169564.90
37637,90590,BEER,MILLER LITE 30PK CAN - 12OZ,MILLER BREWING COMPANY,162971.40
37997,96741,BEER,CORONA EXTRA 4/6 NR - 12OZ,CROWN IMPORTS,140151.32
8205,25883,BEER,MODELO ESPECIAL 24 LOOSE NR - 12OZ,CROWN IMPORTS,126634.80
37632,90468,BEER,BUD LIGHT 30PK CAN,ANHEUSER BUSCH INC,120735.97
37934,96083,BEER,HEINEKEN 4/6 NR - 12OZ,HEINEKEN USA,110962.92
4357,23314,BEER,CORONA EXTRA 18PK NR - 12OZ,CROWN IMPORTS,100913.44


### Monthly Usage Trends for Top Beer Categories

This section analyzes how usage of the most common beer categories changes over time.

At a high level, the code:
1. Filters the dataset to include only items classified as beer  
2. Groups individual beer products into major categories based on item descriptions (e.g., Corona, Bud, Heineken)  
3. Aggregates usage by month for each beer category  
4. Visualizes monthly usage trends to highlight seasonality and relative demand patterns  

The resulting line chart provides insight into which beer categories consistently drive volume and how consumption fluctuates over time, supporting inventory planning, forecasting, and promotional decision-making.


In [ ]:
# filter to only keep beers
df_beers = df[df['item_type']=='BEER']
df_beers = df_beers.copy()
df_beers['beer_type'] = 'other'

corona_mask = df_beers['item_description'].str.contains('corona',case=False,na=False)
df_beers.loc[corona_mask,'beer_type'] = 'corona'

bud_mask = df_beers['item_description'].str.contains('bud',case=False,na=False)
df_beers.loc[bud_mask,'beer_type'] = 'bud'

heineken_mask = df_beers['item_description'].str.contains('heineken',case=False,na=False)
df_beers.loc[heineken_mask,'beer_type'] = 'heineken'

modelo_mask = df_beers['item_description'].str.contains('modelo',case=False,na=False)
df_beers.loc[modelo_mask,'beer_type'] = 'modelo'

miller_mask = df_beers['item_description'].str.contains('miller',case=False,na=False)
df_beers.loc[miller_mask,'beer_type'] = 'miller'

top_beers = df_beers[df_beers['beer_type']!='other']

# plot monthly usage of top beers
monthly_beers = (
    top_beers.groupby(['beer_type',pd.Grouper(key='date',freq='MS')],as_index=False)
    .agg(usage=('usage','sum'))
    .sort_values(['beer_type','date'])
)

fig = px.line(
    monthly_beers,
    x='date',
    y='usage',
    color='beer_type',
    markers=True,
    title='Monthly Usage by Beer'
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Usage',
    legend_title='Beer'
)

fig.show()



## Seasonal Usage Trends and Shifting Market Share

Understanding how product demand evolves over time is essential for forecasting, production planning, and portfolio strategy. In this analysis, I examined monthly usage trends across multiple beer types to identify seasonal patterns and shifts in relative market share within the product lineup.

#### **Seasonal Demand Patterns**
The data shows a strong and recurring seasonal trend. Usage reliably increases during the summer months and declines during the winter, suggesting a clear link between warm-weather activity and beer consumption. This pattern repeats consistently across the entire time horizon, indicating stable seasonality that can be leveraged for operational planning. From an analytics standpoint, this is exactly the type of predictable seasonality that supports accurate forecasting models and capacity planning decisions.

#### **Changes in Market Share Across Beer Types**
While all beer types are affected by seasonality, they do not respond equally. Some beers become significantly more popular during high-traffic summer periods, capturing a larger share of total volume at the peak of the season. Others maintain relatively stable usage throughout the year, acting as “anchor products” that support consistent baseline demand.

These shifts in relative performance highlight how consumer preferences change month-to-month. A beer that is not the top performer in an absolute sense may temporarily overtake others during certain seasons, revealing opportunities for targeted promotions or seasonal releases.

#### **Strategic Implications**
These findings support several practical applications:

- **Inventory and Supply Planning:** Seasonal peaks suggest when to increase production, pre-build inventory, or plan for elevated taproom traffic.
- **Product Positioning:** Beers with strong seasonal surges may benefit from expanded visibility or limited edition variants during peak months.
- **Portfolio Management:** Stable, year-round performers help buffer volatility and provide predictable revenue, making them strategic core offerings.
- **Marketing Timing:** Promotion schedules can be aligned with periods when specific beers historically gain share, improving ROI on marketing spend.

Overall, this analysis moves beyond reporting usage trends and demonstrates how time series insights can inform real decision making. It highlights the ability to identify seasonality, interpret shifting product performance, and connect analytical findings back to business strategy
